In [1]:
import pandas as pd
import numpy as np

In [2]:
# identify number of lines of metadata to skip
def metadata_line_count(file_path):
    with open(file_path, 'r') as file:
        count = 0
        find_str = "#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO"
        # should be way less than 10 thousand lines of metadata
        while count < 10**4:
            line = file.readline().strip()
            if find_str in line:
                return count
            count += 1

In [3]:
def read_vcf(file_path, metadata_line_count):
    df = pd.read_csv(file_path, delimiter='\t', header=0, skiprows=metadata_line_count)
    
    # Split INFO column
    info_split = df['INFO'].str.split(';', expand=True)

    # Rename the new columns with 'INFO_' prefix
    info_split.columns = [f'INFO_{i}' for i in range(1, info_split.shape[1] + 1)]

    # Concatenate the original DataFrame with the new columns
    df = pd.concat([df, info_split], axis=1)
    
    # delete the previous INFO column now that we have all of the split INFO columns
    df = df.drop('INFO', axis=1)
    
    return df

In [4]:
def create_len_col(df, column_name):
    """
    Create a new column containing the lengths of values in a specified column.
    Example use case: create a column for the length of alternative and reference reads
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - column_name (str): Name of the column for which length is calculated.

    Returns:
    - pd.DataFrame: DataFrame with a new column containing lengths of specified values.
    """
    new_col_name = column_name + '_LEN'
    # Only add length column if this column name doesn't already exist
    if new_col_name not in df.columns:
        df[new_col_name] = df[column_name].apply(len)
    return df

In [5]:
def create_snp_id_col(df, column_list, truncate_columns=None):
    """
    Concatenate list of columns in the given order and create a new column as the SNP ID

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - column_list (list): List of column names to concatenate.
    - order of the column names matters ie. CHR, POS, REF, ALT = chr1_2131_A_C vs ALT, REF ...C_A
    Returns:
    - pd.DataFrame: DataFrame with the new concatenated ID column.
    """
    # Ensures all specified columns exist in the DataFrame
    for column in column_list:
        if column not in df.columns:
            raise ValueError(f"Column '{column}' does not exist in the DataFrame.")
    
    # Create a copy of the DataFrame to avoid modifying the original during truncation
    df_copy = df.copy()

    # Truncate each column value if the column is in truncate columns to a maximum length of 5 characters
    for column in column_list:
        if truncate_columns is not None and column in truncate_columns:
            df_copy[column] = df_copy[column].astype(str).apply(lambda x: x[:5])

    
    # Concatenate columns in the specified order
    df['SNP_ID'] = df_copy[column_list].astype(str).agg('_'.join, axis=1)

    return df

In [6]:
def split_select_col(df, column, keyword, new_col_name, multiple_columns, remove_keyword, delimiter='|'):
    split_values = df[column].str.split(delimiter, expand=True)
    found = False
    for col in range(len(split_values.columns)):
        if keyword in split_values.iloc[0][col]:
            if remove_keyword:
                selected_col = split_values.iloc[:, col].str.replace(keyword, '')
            else:
                selected_col = split_values.iloc[:, col]
            selected_col.name = new_col_name
            df = pd.concat([df, selected_col], axis=1)
            print(col)
            found = True
            break
    if not found:
        print('keyword not found in columns')
    return df

In [7]:
# PL stands for "Phred-scaled likelihoods." which is the column of probabilities associated with	GT:AD:DP:GQ:PL
# for now this only extracts DP and AD values
def split_pl(df, column, extract_AD=True, extract_DP=True, extract_alt_read_freq=True):
    # need to extract AD and DP columns if we are extracting ALT_READ frequency
    if extract_alt_read_freq:
        extract_AD, extract_DP = True, True
        
    if extract_AD:
        # Splitting based on ':' delimiter and extracting the 2nd column
        # The second column would be AD from 'GT:AD:DP:GQ:PL'
        df['AD'] = df[column].apply(lambda x: x.split(':')[1])
        # Further splitting the 'AD' column into two columns, left for Alt Reads, right Ref
        split_AD = df['AD'].str.split(',', expand=True)
        df[['REF_READS', 'ALT_READS']] = split_AD.iloc[:, :2]
        # Convert REF_READS, ALT_READS to integer columns
        df['REF_READS'], df['ALT_READS'] = df['REF_READS'].astype(int), df['ALT_READS'].astype(int)
        # Deleting the AD column
        del df['AD']
    
    if extract_DP:
        # Splitting based on ':' delimiter and extracting the 3rd column
        # The second column would be DP from 'GT:AD:DP:GQ:PL'
        df['DP'] = df[column].apply(lambda x: x.split(':')[2])
        df['DP'] = df['DP'].astype(int)
        
    if extract_alt_read_freq:
        # Calculate ALT READ frequency by dividing it by total depth, round to 8 decimal places
        df['ALT_FREQ'] = (df['ALT_READS'] / df['DP']).round(8)

    return df

In [8]:
def filter_variant_type(df, column, keywords, extract=True, extract_all_keywords=False):
    if extract_all_keywords:
        extract = True
        
    for keyword in keywords:
        # create a new column that states whether the variant type of the value contains keyword
        df[keyword + '_exists'] = df[column].str.contains(keyword, case=False)
    if extract:
        # sum of boolean values will determine if at least one is True/one keyword present
        keyword_columns = [keyword + '_exists' for keyword in keywords]
        df['ANY_KEYWORD'] = df[keyword_columns].any(axis=1)
        # if no keywords in variant type, remove the row
        df = df[df['ANY_KEYWORD']]
        del df['ANY_KEYWORD']
    # creating a separate column of all the keywords in that row by concatenating columns
    # for each column turn it into strings column, if it is True change value to keyword else ''
    # then combine all keyword columns, then remove last character which will always be a ','
    # initialize this column with empty strings and iteratively add each column as they are made
    df['VARIANT_TYPE'] = ''
    for keyword in keywords:
        keyword_column_name = keyword +'_exists'
        df[keyword_column_name] = df[keyword_column_name].replace({True: keyword + ',', False: ''})    
        df['VARIANT_TYPE'] = df['VARIANT_TYPE'] + df[keyword_column_name]
        del df[keyword_column_name]
    # specified in parameter extract_all_keywords, just remove the comma at end of all strings
    if extract_all_keywords:
        # removal of the last character which will always be a column since we sum keyword+',' columns
        df['VARIANT_TYPE'] = df['VARIANT_TYPE'].str.slice(0, -1)
    # otherwise, remove everything past the first index of comma, leaving only first value
    else:
        # get first index of comma
        df['COMMA_IDX'] = df['VARIANT_TYPE'].str.find(',').astype('int')
        # remove rest of string after comma, inclusive of comma, i.e. hello, hi --> hello
        df['VARIANT_TYPE'] = df.apply(lambda row: row['VARIANT_TYPE'][:row['COMMA_IDX']], axis=1)
        del df['COMMA_IDX']
    return df

In [108]:
def extract_AA_change(df, column, delimeter='|', truncate=True):
    return df

In [133]:
split_values = df['INFO_19'].str.split('|', expand=True).iloc[:, 4]
print(split_values.str.contains('gene-').all())
print(split_values.str.count('gene-').sum() / len(split_values))

False
0.9956980836918263


In [9]:
# EXAMPLE TEST
file_path = '100M1.final.vcf'
metadata_lines = metadata_line_count(file_path)
df = read_vcf(file_path, metadata_lines)
df = create_len_col(df, 'REF')
df = create_len_col(df, 'ALT')
df = create_snp_id_col(df, ['#CHROM', 'POS', 'REF', 'ALT', 'REF_LEN', 'ALT_LEN'], ['REF', 'ALT'])
df = split_select_col(df, 'INFO_19', 'gene-', 'Gene_Symbol', multiple_columns=False, remove_keyword=True)
df = split_pl(df, '100M1')
filter_variant_keywords = ['disruptive_inframe_deletion', 'disruptive_inframe_insertion', 'frameshift_variant', 'stop_lost', 'stop_gained', 'missense_variant', 'splice_acceptor', 'splice_donor', 'inframe_deletion']
df = filter_variant_type(df, column='INFO_19', keywords=filter_variant_keywords)
df = split_select_col(df, 'INFO_19', 'p.', 'Protein', multiple_columns=True, remove_keyword=False)
display(df)

4
10


/tmp/ipykernel_139023/1273072395.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VARIANT_TYPE'] = ''
/tmp/ipykernel_139023/1273072395.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[keyword_column_name] = df[keyword_column_name].replace({True: keyword + ',', False: ''})
/tmp/ipykernel_139023/1273072395.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,FORMAT,100M1,INFO_1,...,REF_LEN,ALT_LEN,SNP_ID,Gene_Symbol,REF_READS,ALT_READS,DP,ALT_FREQ,VARIANT_TYPE,Protein
0,chr1,3740866,.,TC,T,3487.77,PASS,GT:AD:DP:GQ:PL,"0/1:10,104:114:30:3516,0,30",AC=1,...,2,1,chr1_3740866_TC_T_2_1,Xkr4,10,104,114,0.912281,frameshift_variant,p.Arg235fs
3,chr1,6872919,.,GC,G,4570.77,PASS,GT:AD:DP:GQ:PL,"1/1:8,142:150:99:4599,183,0",AC=2,...,2,1,chr1_6872919_GC_G_2_1,St18,8,142,150,0.946667,frameshift_variant,p.Pro226fs
7,chr1,8677327,.,A,T,4131.77,PASS,GT:AD:DP:GQ:PL,"1/1:19,171:190:47:4160,47,0",AC=2,...,1,1,chr1_8677327_A_T_1_1,Sntg1,19,171,190,0.900000,missense_variant,p.Ser172Arg
10,chr1,9615822,.,C,T,334.77,PASS,GT:AD:DP:GQ:PGT:PID:PL,"0/1:33,11:44:99:0|1:9615802_G_A:363,0,2497",AC=1,...,1,1,chr1_9615822_C_T_1_1,Rrs1,33,11,44,0.250000,missense_variant,p.Thr25Met
11,chr1,9615846,.,A,T,262.77,PASS,GT:AD:DP:GQ:PGT:PID:PL,"0/1:36,10:46:99:0|1:9615802_G_A:291,0,2489",AC=1,...,1,1,chr1_9615846_A_T_1_1,Rrs1,36,10,46,0.217391,missense_variant,p.Glu33Val
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10322,chrX,162979973,.,T,A,1849.77,PASS,GT:AD:DP:GQ:PL,"0/1:33,74:107:99:1878,0,606",AC=1,...,1,1,chrX_162979973_T_A_1_1,Bmx,33,74,107,0.691589,missense_variant,p.Lys604Met
10325,chrX,163739523,.,CA,C,998.77,PASS,GT:AD:DP:GQ:PL,"0/1:89,58:160:99:1027,0,1777",AC=1,...,2,1,chrX_163739523_CA_C_2_1,Mospd2,89,58,160,0.362500,splice_acceptor,
10338,chrX,166090512,.,A,T,1729.77,PASS,GT:AD:DP:GQ:PL,"0/1:97,73:170:99:1758,0,2595",AC=1,...,1,1,chrX_166090512_A_T_1_1,Tlr7,97,73,170,0.429412,missense_variant,p.Tyr328Asn
10339,chrX,166260331,.,T,A,2804.77,PASS,GT:AD:DP:GQ:PL,"0/1:75,103:178:99:2833,0,1717",AC=1,...,1,1,chrX_166260331_T_A_1_1,Frmpd4,75,103,178,0.578652,missense_variant,p.Asp1155Val


In [11]:
line2 = """#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	100M1"""
line = """chr1	3740866	.	TC	T	3487.77	PASS	AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-3.460e-01;ClippingRankSum=0.426;DP=116;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=60.19;MQRankSum=1.32;QD=30.59;ReadPosRankSum=0.175;SOR=0.625;set=variant2;DEL;HET;VARTYPE=DEL;ANN=T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-NM_001011874.1|Coding|1/3|c.705delG|p.Arg235fs|855/3634|705/1944|235/647||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_006495550.5|Coding|1/4|c.705delG|p.Arg235fs|867/12274|705/1944|235/647||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_011238395.4|Coding|1/3|c.705delG|p.Arg235fs|867/2104|705/1011|235/336||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_036152287.1|Coding|1/3|c.705delG|p.Arg235fs|867/2764|705/1026|235/341||,T|downstream_gene_variant|MODIFIER|Gene_gene-LOC108167595|Gene_gene-LOC108167595|transcript|gene-LOC108167595|Noncoding||n.*969delG|||||866|;LOF=(Xkr4|gene-Xkr4|4|1.00)	GT:AD:DP:GQ:PL	0/1:10,104:114:30:3516,0,30"""

In [12]:
print(line.split())

['chr1', '3740866', '.', 'TC', 'T', '3487.77', 'PASS', 'AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-3.460e-01;ClippingRankSum=0.426;DP=116;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=60.19;MQRankSum=1.32;QD=30.59;ReadPosRankSum=0.175;SOR=0.625;set=variant2;DEL;HET;VARTYPE=DEL;ANN=T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-NM_001011874.1|Coding|1/3|c.705delG|p.Arg235fs|855/3634|705/1944|235/647||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_006495550.5|Coding|1/4|c.705delG|p.Arg235fs|867/12274|705/1944|235/647||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_011238395.4|Coding|1/3|c.705delG|p.Arg235fs|867/2104|705/1011|235/336||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_036152287.1|Coding|1/3|c.705delG|p.Arg235fs|867/2764|705/1026|235/341||,T|downstream_gene_variant|MODIFIER|Gene_gene-LOC108167595|Gene_gene-LOC108167595|transcript|gene-LOC108167595|Noncoding||n.*969delG|||||866|;LOF=(Xkr4|gene-Xkr4|4|1.00)', 'GT:AD:DP:GQ:PL', '0/1:10,104:114:30:3

In [9]:
file_path = '222511.hg38_multianno.txt'

with open(file_path, 'r') as file:
    while True:
        x = input()
        if x == "":
            print(file.readline().strip())
        else:
            break


Chr	Start	End	Ref	Alt	abraom_freq	abraom_filter	abraom_cegh_filter	ALL.sites.2015_08	avsnp150	CLNALLELEID	CLNDN	CLNDISDB	CLNREVSTAT	CLNSIG	cosmic70	Start_hg19	DamagePredCount	SIFT_pred	SIFT4G_pred	LRT_pred	MutationTaster_pred	MutationAssessor_pred	FATHMM_pred	PROVEAN_pred	MetaSVM_pred	MetaLR_pred	M-CAP_pred	MutPred_score	MVP_score	MPC_score	PrimateAI_pred	DEOGEN2_pred	BayesDel_addAF_pred	BayesDel_noAF_pred	LIST-S2_pred	DANN_score	fathmm-MKL_coding_pred	fathmm-XF_coding_pred	Eigen-raw_coding	Eigen-phred_coding	Eigen-PC-raw_coding	Eigen-PC-phred_coding	integrated_fitCons_score	GM12878_fitCons_score	H1-hESC_fitCons_score	HUVEC_fitCons_score	GERP++_NR	GERP++_RS	phyloP100way_vertebrate	phyloP30way_mammalian	phyloP17way_primate	phastCons100way_vertebrate	phastCons30way_mammalian	phastCons17way_primate	bStatistic	Interpro_domain	GTEx_V8_gene	GTEx_V8_tissue	esp6500siv2_all	ExAC_ALL	ExAC_AFR	ExAC_AMR	ExAC_EAS	ExAC_FIN	ExAC_NFE	ExAC_OTH	ExAC_SAS	DN ID	Patient ID	Phenotype	Platform	Study	Pubmed 

In [43]:
import pandas as pd

# Replace 'your_file.txt' with the actual path to your file
file_path = '100M1.final.vcf'

# Specify the delimiter (space in this case)
delimiter = ' '

# Number of lines to skip at the beginning
skiprows = 0

# Open the file and read lines until a line with the correct delimiter is found
with open(file_path, 'r') as file:
    while True:
        line = file.readline()
        skiprows += 1

        # Check if the line contains the expected delimiter
        if 'GT:AD:DP:GQ:PL' in line:
            break

# Read the text file into a DataFrame, skipping the lines until the delimiter is found
df = pd.read_csv(file_path, delimiter, header=None, skiprows=skiprows-6)

# Display the DataFrame
print(df)
print(skiprows)

                                                       0
0      #CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\...
1      chr1\t3740866\t.\tTC\tT\t3487.77\tPASS\tAC=1;S...
2      chr1\t6304093\t.\tTTG\tT\t31.77\tMG_INDEL_Filt...
3      chr1\t6841143\t.\tCT\tC\t75.77\tMG_INDEL_Filte...
4      chr1\t6872919\t.\tGC\tG\t4570.77\tPASS\tAC=2;S...
...                                                  ...
10339  chrX\t166090512\t.\tA\tT\t1729.77\tPASS\tAC=1;...
10340  chrX\t166260331\t.\tT\tA\t2804.77\tPASS\tAC=1;...
10341  chrX\t167579241\t.\tA\tT\t604.77\tPASS\tAC=1;S...
10342  chrX\t168084638\t.\tT\tC\t218.77\tPASS\tAC=1;S...
10343  chrX\t168559972\t.\tG\tT\t1803.77\tPASS\tAC=1;...

[10344 rows x 1 columns]
150


/tmp/ipykernel_373592/1067633238.py:23: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(file_path, delimiter, header=None, skiprows=skiprows-6)


In [44]:
df = df[0].str.split('\t', expand=True)

In [45]:
display(df)

,0,1,2,3,4,5,6,7,8,9
0,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,100M1
1,chr1,3740866,.,TC,T,3487.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-3.460...,GT:AD:DP:GQ:PL,"0/1:10,104:114:30:3516,0,30"
2,chr1,6304093,.,TTG,T,31.77,MG_INDEL_Filter,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-1.391...,GT:AD:DP:GQ:PL,"0/1:30,5:35:60:60,0,838"
3,chr1,6841143,.,CT,C,75.77,MG_INDEL_Filter,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=0.727;...,GT:AD:DP:GQ:PL,"0/1:45,11:58:99:104,0,830"
4,chr1,6872919,.,GC,G,4570.77,PASS,AC=2;SAMPLES_AF=1.00;AN=2;BaseQRankSum=0.130;C...,GT:AD:DP:GQ:PL,"1/1:8,142:150:99:4599,183,0"
...,...,...,...,...,...,...,...,...,...,...
10339,chrX,166090512,.,A,T,1729.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-5.350...,GT:AD:DP:GQ:PL,"0/1:97,73:170:99:1758,0,2595"
10340,chrX,166260331,.,T,A,2804.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=1.91;C...,GT:AD:DP:GQ:PL,"0/1:75,103:178:99:2833,0,1717"
10341,chrX,167579241,.,A,T,604.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-1.100...,GT:AD:DP:GQ:PL,"0/1:40,29:69:99:633,0,1024"
10342,chrX,168084638,.,T,C,218.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=3.57;C...,GT:AD:DP:GQ:PGT:PID:PL,"0/1:13,7:20:99:0|1:168084632_G_T:247,0,342"


In [49]:
df.to_csv('')

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,100M1
0,chr1,3740866,.,TC,T,3487.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-3.460...,GT:AD:DP:GQ:PL,"0/1:10,104:114:30:3516,0,30"
1,chr1,6304093,.,TTG,T,31.77,MG_INDEL_Filter,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-1.391...,GT:AD:DP:GQ:PL,"0/1:30,5:35:60:60,0,838"
2,chr1,6841143,.,CT,C,75.77,MG_INDEL_Filter,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=0.727;...,GT:AD:DP:GQ:PL,"0/1:45,11:58:99:104,0,830"
3,chr1,6872919,.,GC,G,4570.77,PASS,AC=2;SAMPLES_AF=1.00;AN=2;BaseQRankSum=0.130;C...,GT:AD:DP:GQ:PL,"1/1:8,142:150:99:4599,183,0"
4,chr1,6878048,.,AT,A,53.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-3.400...,GT:AD:DP:GQ:PL,"0/1:11,5:16:82:82,0,224"
...,...,...,...,...,...,...,...,...,...,...
10338,chrX,166090512,.,A,T,1729.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-5.350...,GT:AD:DP:GQ:PL,"0/1:97,73:170:99:1758,0,2595"
10339,chrX,166260331,.,T,A,2804.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=1.91;C...,GT:AD:DP:GQ:PL,"0/1:75,103:178:99:2833,0,1717"
10340,chrX,167579241,.,A,T,604.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-1.100...,GT:AD:DP:GQ:PL,"0/1:40,29:69:99:633,0,1024"
10341,chrX,168084638,.,T,C,218.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=3.57;C...,GT:AD:DP:GQ:PGT:PID:PL,"0/1:13,7:20:99:0|1:168084632_G_T:247,0,342"


In [46]:
df.columns = df.iloc[0]

In [57]:
df.to_csv('delimited_100M1_final.csv', index=False)

In [48]:
df = df.drop(0)
df = df.reset_index(drop=True)

In [34]:
file_path = '100M1.final.vcf'  # Replace with the actual path to your file

# Open the file and count the lines
with open(file_path, 'r') as file:
    line_count = sum(1 for line in file)

print(f'The file has {line_count} lines.')

The file has 10492 lines.


In [39]:
df.to_csv('test.csv')

In [52]:
print(df['INFO'].iloc[0])

AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-3.460e-01;ClippingRankSum=0.426;DP=116;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=60.19;MQRankSum=1.32;QD=30.59;ReadPosRankSum=0.175;SOR=0.625;set=variant2;DEL;HET;VARTYPE=DEL;ANN=T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-NM_001011874.1|Coding|1/3|c.705delG|p.Arg235fs|855/3634|705/1944|235/647||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_006495550.5|Coding|1/4|c.705delG|p.Arg235fs|867/12274|705/1944|235/647||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_011238395.4|Coding|1/3|c.705delG|p.Arg235fs|867/2104|705/1011|235/336||,T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|transcript|rna-XM_036152287.1|Coding|1/3|c.705delG|p.Arg235fs|867/2764|705/1026|235/341||,T|downstream_gene_variant|MODIFIER|Gene_gene-LOC108167595|Gene_gene-LOC108167595|transcript|gene-LOC108167595|Noncoding||n.*969delG|||||866|;LOF=(Xkr4|gene-Xkr4|4|1.00)


In [53]:
info_split = df['INFO'].str.split(';', expand=True)

# Rename the new columns with 'INFO_' prefix
info_split.columns = [f'INFO_{i}' for i in range(1, info_split.shape[1] + 1)]

# Concatenate the original DataFrame with the new columns
df = pd.concat([df, info_split], axis=1)

In [54]:
df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,100M1,...,INFO_12,INFO_13,INFO_14,INFO_15,INFO_16,INFO_17,INFO_18,INFO_19,INFO_20,INFO_21
0,chr1,3740866,.,TC,T,3487.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-3.460...,GT:AD:DP:GQ:PL,"0/1:10,104:114:30:3516,0,30",...,QD=30.59,ReadPosRankSum=0.175,SOR=0.625,set=variant2,DEL,HET,VARTYPE=DEL,ANN=T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|t...,LOF=(Xkr4|gene-Xkr4|4|1.00),None
1,chr1,6304093,.,TTG,T,31.77,MG_INDEL_Filter,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-1.391...,GT:AD:DP:GQ:PL,"0/1:30,5:35:60:60,0,838",...,QD=0.91,ReadPosRankSum=0.535,SOR=0.313,set=FilteredInAll,DEL,HET,VARTYPE=DEL,ANN=T|intron_variant|MODIFIER|Rb1cc1|gene-Rb1c...,None,None
2,chr1,6841143,.,CT,C,75.77,MG_INDEL_Filter,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=0.727;...,GT:AD:DP:GQ:PL,"0/1:45,11:58:99:104,0,830",...,QD=1.35,ReadPosRankSum=0.815,SOR=0.725,set=FilteredInAll,DEL,HET,VARTYPE=DEL,ANN=C|intron_variant|MODIFIER|St18|gene-St18|t...,None,None
3,chr1,6872919,.,GC,G,4570.77,PASS,AC=2;SAMPLES_AF=1.00;AN=2;BaseQRankSum=0.130;C...,GT:AD:DP:GQ:PL,"1/1:8,142:150:99:4599,183,0",...,QD=30.47,ReadPosRankSum=0.163,SOR=0.446,set=variant2,DEL,HOM,VARTYPE=DEL,ANN=G|frameshift_variant|HIGH|St18|gene-St18|t...,LOF=(St18|gene-St18|29|0.93),None
4,chr1,6878048,.,AT,A,53.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-3.400...,GT:AD:DP:GQ:PL,"0/1:11,5:16:82:82,0,224",...,QD=3.36,ReadPosRankSum=0.566,SOR=0.223,set=variant2,DEL,HET,VARTYPE=DEL,ANN=A|intron_variant|MODIFIER|St18|gene-St18|t...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10338,chrX,166090512,.,A,T,1729.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-5.350...,GT:AD:DP:GQ:PL,"0/1:97,73:170:99:1758,0,2595",...,QD=10.18,ReadPosRankSum=1.40,SOR=0.923,set=variant,SNP,HET,VARTYPE=SNP,ANN=T|missense_variant|MODERATE|Tlr7|gene-Tlr7...,None,None
10339,chrX,166260331,.,T,A,2804.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=1.91;C...,GT:AD:DP:GQ:PL,"0/1:75,103:178:99:2833,0,1717",...,QD=15.76,ReadPosRankSum=4.02,SOR=0.718,set=variant,SNP,HET,VARTYPE=SNP,ANN=A|missense_variant|MODERATE|Frmpd4|gene-Fr...,None,None
10340,chrX,167579241,.,A,T,604.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=-1.100...,GT:AD:DP:GQ:PL,"0/1:40,29:69:99:633,0,1024",...,QD=8.76,ReadPosRankSum=2.50,SOR=0.604,set=variant,SNP,HET,VARTYPE=SNP,ANN=T|missense_variant|MODERATE|Arhgap6|gene-A...,None,None
10341,chrX,168084638,.,T,C,218.77,PASS,AC=1;SAMPLES_AF=0.500;AN=2;BaseQRankSum=3.57;C...,GT:AD:DP:GQ:PGT:PID:PL,"0/1:13,7:20:99:0|1:168084632_G_T:247,0,342",...,QD=10.94,ReadPosRankSum=-7.920e-01,SOR=1.402,set=variant,SNP,HET,VARTYPE=SNP,ANN=C|intron_variant|MODIFIER|Arhgap6|gene-Arh...,None,None


In [55]:
df = df.drop('INFO', axis=1)

In [56]:
df

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,FORMAT,100M1,INFO_1,...,INFO_12,INFO_13,INFO_14,INFO_15,INFO_16,INFO_17,INFO_18,INFO_19,INFO_20,INFO_21
0,chr1,3740866,.,TC,T,3487.77,PASS,GT:AD:DP:GQ:PL,"0/1:10,104:114:30:3516,0,30",AC=1,...,QD=30.59,ReadPosRankSum=0.175,SOR=0.625,set=variant2,DEL,HET,VARTYPE=DEL,ANN=T|frameshift_variant|HIGH|Xkr4|gene-Xkr4|t...,LOF=(Xkr4|gene-Xkr4|4|1.00),None
1,chr1,6304093,.,TTG,T,31.77,MG_INDEL_Filter,GT:AD:DP:GQ:PL,"0/1:30,5:35:60:60,0,838",AC=1,...,QD=0.91,ReadPosRankSum=0.535,SOR=0.313,set=FilteredInAll,DEL,HET,VARTYPE=DEL,ANN=T|intron_variant|MODIFIER|Rb1cc1|gene-Rb1c...,None,None
2,chr1,6841143,.,CT,C,75.77,MG_INDEL_Filter,GT:AD:DP:GQ:PL,"0/1:45,11:58:99:104,0,830",AC=1,...,QD=1.35,ReadPosRankSum=0.815,SOR=0.725,set=FilteredInAll,DEL,HET,VARTYPE=DEL,ANN=C|intron_variant|MODIFIER|St18|gene-St18|t...,None,None
3,chr1,6872919,.,GC,G,4570.77,PASS,GT:AD:DP:GQ:PL,"1/1:8,142:150:99:4599,183,0",AC=2,...,QD=30.47,ReadPosRankSum=0.163,SOR=0.446,set=variant2,DEL,HOM,VARTYPE=DEL,ANN=G|frameshift_variant|HIGH|St18|gene-St18|t...,LOF=(St18|gene-St18|29|0.93),None
4,chr1,6878048,.,AT,A,53.77,PASS,GT:AD:DP:GQ:PL,"0/1:11,5:16:82:82,0,224",AC=1,...,QD=3.36,ReadPosRankSum=0.566,SOR=0.223,set=variant2,DEL,HET,VARTYPE=DEL,ANN=A|intron_variant|MODIFIER|St18|gene-St18|t...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10338,chrX,166090512,.,A,T,1729.77,PASS,GT:AD:DP:GQ:PL,"0/1:97,73:170:99:1758,0,2595",AC=1,...,QD=10.18,ReadPosRankSum=1.40,SOR=0.923,set=variant,SNP,HET,VARTYPE=SNP,ANN=T|missense_variant|MODERATE|Tlr7|gene-Tlr7...,None,None
10339,chrX,166260331,.,T,A,2804.77,PASS,GT:AD:DP:GQ:PL,"0/1:75,103:178:99:2833,0,1717",AC=1,...,QD=15.76,ReadPosRankSum=4.02,SOR=0.718,set=variant,SNP,HET,VARTYPE=SNP,ANN=A|missense_variant|MODERATE|Frmpd4|gene-Fr...,None,None
10340,chrX,167579241,.,A,T,604.77,PASS,GT:AD:DP:GQ:PL,"0/1:40,29:69:99:633,0,1024",AC=1,...,QD=8.76,ReadPosRankSum=2.50,SOR=0.604,set=variant,SNP,HET,VARTYPE=SNP,ANN=T|missense_variant|MODERATE|Arhgap6|gene-A...,None,None
10341,chrX,168084638,.,T,C,218.77,PASS,GT:AD:DP:GQ:PGT:PID:PL,"0/1:13,7:20:99:0|1:168084632_G_T:247,0,342",AC=1,...,QD=10.94,ReadPosRankSum=-7.920e-01,SOR=1.402,set=variant,SNP,HET,VARTYPE=SNP,ANN=C|intron_variant|MODIFIER|Arhgap6|gene-Arh...,None,None
